## Acquire all comments

In [1]:
import praw
import nltk
import pandas as pd

reddit = praw.Reddit(user_agent='Comment Extraction (by /u/Combatmeatball)',
                     client_id='4EHtFfPEuH2nYg', client_secret="tLbFWv3A6Bz5OQGj2V3id40XEkw",
                     username='combatmeatball', password='cdgf123')


In [21]:
win1 = reddit.submission(id='eafro9') #kings 12/13
win2 = reddit.submission(id='e9jsyu') #warriors 12/11
win3 = reddit.submission(id='dtqy15') #mavs 11/8
win4 = reddit.submission(id='dwl4fv') #mavs 11/14
win5 = reddit.submission(id='dye0h5') #cavs 11/18
win6 = reddit.submission(id='doix9q') #bulls 10/28
win7 = reddit.submission(id='ec6kou') #hawks 12/17
win8 = reddit.submission(id='eg57e4') #nets 12/26

lost1 = reddit.submission(id='eb9f8l') #nuggets 12/15
lost2 = reddit.submission(id='e92ohs') #blazers 12/10
lost3 = reddit.submission(id='e7o2gg') #pacers 12/7
lost4 = reddit.submission(id='e6sblh') #nuggets 12/5
lost5 = reddit.submission(id='e4pfyt') #celtics 12/1
lost6 = reddit.submission(id='e5affc') #bucks 12/2
lost7 = reddit.submission(id='e3ptov') #sixers 11/29
lost8 = reddit.submission(id='eeuqso') #wizards 12/23
lost9 = reddit.submission(id='edkl1r') #heat 12/20
lost10 = reddit.submission(id='draeuh') #kings 11/3

win_threads = [win1,win2,win3,win4,win5,win6,win7,win8]
loss_threads = [lost1,lost2,lost3,lost4,lost5,lost6,lost7,lost8,lost9,lost10]

In [22]:
win_comments = []
lose_comments = []
for win in win_threads:
    win.comments.replace_more(limit=0)
    for comment in win.comments.list():
        win_comments.append(comment.body)
for loss in loss_threads:
    loss.comments.replace_more(limit=0)
    for comment in loss.comments.list():
        lose_comments.append(comment.body)

In [23]:
winDf = pd.DataFrame(win_comments,columns = ['comment'])
loseDf = pd.DataFrame(lose_comments,columns = ['comment'])
winDf['label'] = 'win'
loseDf['label'] = 'lose'


In [24]:
df_all = pd.concat([winDf, loseDf])
df_all.head()

,comment,label
0,Nobody:\n\n&nbsp;\n\nMike Miller: 🙂,win
1,🚨**THE NEW YORK KNICKS HAVE A HEAD COACH**🚨,win
2,Mike Miller win streaks in 2019: 1\n\nDavid Fi...,win
3,You know what Mike Miller represents. Work and...,win
4,Miller has recognized that this team can only ...,win


In [26]:
print("Out of {} rows, {} are wins, {} are losses".format(len(df_all),
                                                       len(df_all[df_all['label']=='win']),
                                                       len(df_all[df_all['label']=='lose'])))

Out of 4946 rows, 2383 are wins, 2563 are losses


## Data Cleaning
### Removing 

In [27]:
import regex as re
import string
# helper function to clean tweets
def process(text):
    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    #Convert @username to AT_USER
    text = re.sub('@[^\s]+','',text)
    # Remove tickers
    text = re.sub(r'\$\w*', '', text)
    # To lowercase
    text = text.lower()
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    # Remove hashtags
    text = re.sub(r'#\w*', '', text)
    # Remove Punctuation and split 's, 't, 've with a space for filter
    text = re.sub(r'[' + string.punctuation.replace('@', '') + ']+', ' ', text)
    # Remove words with 2 or fewer letters
    text = re.sub(r'\b\w{1,2}\b', '', text)
    # Remove whitespace (including new line characters)
    text = re.sub(r'\s\s+', ' ', text)
    # Remove single space remaining at the front of the tweet.
    text = text.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    text = ''.join(c for c in text if c <= '\uFFFF') 
    return text

df_all['comment_clean'] = df_all['comment'].apply(process)

df_all.head()

,comment,label,comment_clean
0,Nobody:\n\n&nbsp;\n\nMike Miller: 🙂,win,nobody mike miller
1,🚨**THE NEW YORK KNICKS HAVE A HEAD COACH**🚨,win,the new york knicks have head coach
2,Mike Miller win streaks in 2019: 1\n\nDavid Fi...,win,mike miller win streaks 2019 david fizdale win...
3,You know what Mike Miller represents. Work and...,win,you know what mike miller represents work and ...
4,Miller has recognized that this team can only ...,win,miller has recognized that this team can only ...


In [28]:
stopwords = nltk.corpus.stopwords.words('english')
def tokenize(text):
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

df_all['tokenized'] = df_all['comment_clean'].apply(tokenize)
df_all.head()


,comment,label,comment_clean,tokenized
0,Nobody:\n\n&nbsp;\n\nMike Miller: 🙂,win,nobody mike miller,"[nobody, mike, miller, ]"
1,🚨**THE NEW YORK KNICKS HAVE A HEAD COACH**🚨,win,the new york knicks have head coach,"[, new, york, knicks, head, coach, ]"
2,Mike Miller win streaks in 2019: 1\n\nDavid Fi...,win,mike miller win streaks 2019 david fizdale win...,"[mike, miller, win, streaks, 2019, david, fizd..."
3,You know what Mike Miller represents. Work and...,win,you know what mike miller represents work and ...,"[know, mike, miller, represents, work, home, g..."
4,Miller has recognized that this team can only ...,win,miller has recognized that this team can only ...,"[miller, recognized, team, thrive, payton, rob..."


In [29]:
df_all.to_csv("Redditcomments_cleaned.csv", sep=',')

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=2000)
vectorized = vectorizer.fit_transform(df_all['comment_clean']).toarray()

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectorized, df_all['label'], test_size=0.2, random_state=42)

In [35]:
from sklearn.ensemble import RandomForestClassifier

win_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
win_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [36]:
predictions = win_classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[333 167]
 [197 293]]
              precision    recall  f1-score   support

        lose       0.63      0.67      0.65       500
         win       0.64      0.60      0.62       490

    accuracy                           0.63       990
   macro avg       0.63      0.63      0.63       990
weighted avg       0.63      0.63      0.63       990

0.6323232323232323
